Akansha Patel DS6A 2003 Assignment 3

In [ ]:
!git clone https://github.com/makhan010385/Soybean-.git

Cloning into 'Soybean-'...
remote: Enumerating objects: 404, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 404 (delta 1), reused 53 (delta 0), pack-reused 347
Receiving objects: 100% (404/404), 1.75 GiB | 26.84 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Updating files: 100% (355/355), done.


#LeNet


Load and Preprocess Data

In [ ]:
import numpy as np
import os
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# Define the diseases
diseases = ['Bacterial Pustule', 'Frogeye Leaf Spot', 'Healthy', 'Rust', 'Sudden Death Syndrome', 'Target Leaf Spot', 'Yellow Mosaic']

# Initialize lists to store image data and labels
images = []
labels = []

# Define data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and augment images
for idx, disease in enumerate(diseases):
    disease_path = os.path.join('/content/Soybean-/Soybean Leaf Dataset for Disease Classification', disease)  # Replace with the path to your dataset
    image_files = [os.path.join(disease_path, f) for f in os.listdir(disease_path) if f.endswith('.jpg')]

    # Split the images into training and validation sets (40 images) and testing set (10 images)
    train_val_images = image_files[:40]
    test_images = image_files[40:]

    for image_file in train_val_images:
        image = load_img(image_file, target_size=(128, 128))  # Resize image to 224x224
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)  # Expand dimensions to (1, height, width, channels) for flow() method

        # Generate augmented images
        for batch in datagen.flow(image, batch_size=1):
            augmented_image = batch[0]
            images.append(augmented_image)
            labels.append(idx)
            break  # Break the loop after one augmentation to avoid infinite loop

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Shuffle the data
shuffle_idx = np.random.permutation(len(images))
images = images[shuffle_idx]
labels = labels[shuffle_idx]

# Normalize the images
images = images / 255.0

# Split the dataset into training, validation, and testing sets
X_train_val, X_test, y_train_val, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def create_lenet_model(input_shape):
    model = Sequential()

    # First Convolutional Layer
    model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Second Convolutional Layer
    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(120, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(len(diseases), activation='softmax'))

    return model

input_shape = images[0].shape
model = create_lenet_model(input_shape)


Compile Keras Model

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Fit Keras Model

In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stop]
)


Epoch 1/100
6/6 [==============================] - 3s 399ms/step - loss: 1.8364 - accuracy: 0.2123 - val_loss: 1.7957 - val_accuracy: 0.3556
Epoch 2/100
6/6 [==============================] - 2s 377ms/step - loss: 1.6462 - accuracy: 0.3966 - val_loss: 1.7574 - val_accuracy: 0.2444
Epoch 3/100
6/6 [==============================] - 2s 360ms/step - loss: 1.5314 - accuracy: 0.3855 - val_loss: 1.7699 - val_accuracy: 0.2222
Epoch 4/100
6/6 [==============================] - 3s 485ms/step - loss: 1.4054 - accuracy: 0.4302 - val_loss: 1.7336 - val_accuracy: 0.3333
Epoch 5/100
6/6 [==============================] - 3s 466ms/step - loss: 1.2502 - accuracy: 0.5754 - val_loss: 1.6790 - val_accuracy: 0.3778
Epoch 6/100
6/6 [==============================] - 2s 353ms/step - loss: 1.0469 - accuracy: 0.6592 - val_loss: 1.8903 - val_accuracy: 0.2667
Epoch 7/100
6/6 [==============================] - 2s 320ms/step - loss: 0.9777 - accuracy: 0.6480 - val_loss: 1.7301 - val_accuracy: 0.3556
Epoch 8/100
6

Evaluate Keras Model

In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy*100:.2f}%")


2/2 [==============================] - 0s 57ms/step - loss: 1.6790 - accuracy: 0.3778
Validation Loss: 1.6790
Validation Accuracy: 37.78%


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

2/2 [==============================] - 0s 82ms/step - loss: 1.2678 - accuracy: 0.4464
Test Loss: 1.2678
Test Accuracy: 44.64%


Identify Disease from Test Images

In [ ]:
def predict_disease(image_path):
    image = load_img(image_path, target_size=(128, 128))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0) / 255.0
    prediction = model.predict(image)
    predicted_label = np.argmax(prediction)
    return diseases[predicted_label]

# Example usage
image_path = '/content/Soybean-/Soybean Leaf Dataset for Disease Classification/Target Leaf Spot/Target LS (48).jpg'  # Replace with the path to your test image
predicted_disease = predict_disease(image_path)
print(f"Predicted Disease: {predicted_disease}")


1/1 [==============================] - 0s 113ms/step
Predicted Disease: Target Leaf Spot


In [ ]:
folder_path = '/content/Soybean-/Soybean Leaf Dataset for Disease Classification/Rust'  # Replace with the path to your folder

# List all files in the folder
files = os.listdir(folder_path)

# Count the number of files
num_files = len(files)
print(num_files)



51


#AlexNet


Load and Preprocess Data

In [ ]:
import numpy as np
import os
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# Define the diseases
diseases = ['Bacterial Pustule', 'Frogeye Leaf Spot', 'Healthy', 'Rust', 'Sudden Death Syndrome', 'Target Leaf Spot', 'Yellow Mosaic']

# Initialize lists to store image data and labels
images = []
labels = []

# Define data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and augment images
for idx, disease in enumerate(diseases):
    disease_path = os.path.join('/content/Soybean-/Soybean Leaf Dataset for Disease Classification', disease)  # Replace with the path to your dataset
    image_files = [os.path.join(disease_path, f) for f in os.listdir(disease_path) if f.endswith('.jpg')]

    # Split the images into training and validation sets (40 images) and testing set (10 images)
    train_val_images = image_files[:40]
    test_images = image_files[40:]

    for image_file in train_val_images:
        image = load_img(image_file, target_size=(227, 227))  # Resize image to 227x227
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)  # Expand dimensions to (1, height, width, channels) for flow() method

        # Generate augmented images
        for batch in datagen.flow(image, batch_size=1):
            augmented_image = batch[0]
            images.append(augmented_image)
            labels.append(idx)
            break  # Break the loop after one augmentation to avoid infinite loop

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Shuffle the data
shuffle_idx = np.random.permutation(len(images))
images = images[shuffle_idx]
labels = labels[shuffle_idx]

# Normalize the images
images = images / 255.0

# Split the dataset into training, validation, and testing sets
X_train_val, X_test, y_train_val, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)


Define AlexNet Model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_alexnet_model(input_shape):
    model = Sequential()

    # First Convolutional Layer
    model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Second Convolutional Layer
    model.add(Conv2D(256, kernel_size=(5, 5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Third Convolutional Layer
    model.add(Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'))

    # Fourth Convolutional Layer
    model.add(Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'))

    # Fifth Convolutional Layer
    model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(diseases), activation='softmax'))

    return model

input_shape = images[0].shape
model = create_alexnet_model(input_shape)


Compile Keras Model

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Fit Keras Model

In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stop]
)


Epoch 1/100
6/6 [==============================] - 32s 5s/step - loss: 2.2472 - accuracy: 0.1397 - val_loss: 1.9613 - val_accuracy: 0.0667
Epoch 2/100
6/6 [==============================] - 30s 5s/step - loss: 1.9460 - accuracy: 0.1117 - val_loss: 1.9644 - val_accuracy: 0.1111
Epoch 3/100
6/6 [==============================] - 34s 6s/step - loss: 1.9540 - accuracy: 0.1844 - val_loss: 1.9476 - val_accuracy: 0.1556
Epoch 4/100
6/6 [==============================] - 28s 5s/step - loss: 1.9363 - accuracy: 0.1788 - val_loss: 1.9644 - val_accuracy: 0.0667
Epoch 5/100
6/6 [==============================] - 30s 5s/step - loss: 1.9386 - accuracy: 0.1844 - val_loss: 1.9727 - val_accuracy: 0.1111
Epoch 6/100
6/6 [==============================] - 31s 5s/step - loss: 1.9353 - accuracy: 0.1676 - val_loss: 1.9801 - val_accuracy: 0.1111
Epoch 7/100
6/6 [==============================] - 30s 5s/step - loss: 1.9339 - accuracy: 0.1732 - val_loss: 1.9902 - val_accuracy: 0.1111
Epoch 8/100
6/6 [==========

Evaluate Keras Model

In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy*100:.2f}%")


2/2 [==============================] - 2s 446ms/step - loss: 1.4787 - accuracy: 0.3778
Validation Loss: 1.4787
Validation Accuracy: 37.78%


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

2/2 [==============================] - 2s 798ms/step - loss: 1.6631 - accuracy: 0.3036
Test Loss: 1.6631
Test Accuracy: 30.36%


Identify Disease from Test Images

In [ ]:
def predict_disease(image_path):
    image = load_img(image_path, target_size=(227, 227))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0) / 255.0
    prediction = model.predict(image)
    predicted_label = np.argmax(prediction)
    return diseases[predicted_label]

# Example usage
test_image_path = '/content/Soybean-/Soybean Leaf Dataset for Disease Classification/Rust/Rust (46).jpg'  # Replace with the path to your test image
predicted_disease = predict_disease(test_image_path)
print(f"Predicted Disease: {predicted_disease}")


1/1 [==============================] - 0s 158ms/step
Predicted Disease: Yellow Mosaic


#VGG


Load and Preprocess Data

In [ ]:
import numpy as np
import os
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# Define the diseases
diseases = ['Bacterial Pustule', 'Frogeye Leaf Spot', 'Healthy', 'Rust', 'Sudden Death Syndrome', 'Target Leaf Spot', 'Yellow Mosaic']

# Initialize lists to store image data and labels
images = []
labels = []

# Define data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and augment images
for idx, disease in enumerate(diseases):
    disease_path = os.path.join('/content/Soybean-/Soybean Leaf Dataset for Disease Classification', disease)  # Replace with the path to your dataset
    image_files = [os.path.join(disease_path, f) for f in os.listdir(disease_path) if f.endswith('.jpg')]

    # Split the images into training and validation sets (40 images) and testing set (10 images)
    train_val_images = image_files[:40]
    test_images = image_files[40:]

    for image_file in train_val_images:
        image = load_img(image_file, target_size=(224, 224))  # Resize image to 224x224
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)  # Expand dimensions to (1, height, width, channels) for flow() method

        # Generate augmented images
        for batch in datagen.flow(image, batch_size=1):
            augmented_image = batch[0]
            images.append(augmented_image)
            labels.append(idx)
            break  # Break the loop after one augmentation to avoid infinite loop

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Shuffle the data
shuffle_idx = np.random.permutation(len(images))
images = images[shuffle_idx]
labels = labels[shuffle_idx]

# Normalize the images
images = images / 255.0

# Split the dataset into training, validation, and testing sets
X_train_val, X_test, y_train_val, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)


Define VGG16 Model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.applications import VGG16

def create_vgg16_model(input_shape):
    vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    model = Sequential()
    model.add(vgg16_model)

    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(len(diseases), activation='softmax'))

    return model

input_shape = (224, 224, 3)  # VGG16 input shape
model = create_vgg16_model(input_shape)


58889256/58889256 [==============================] - 0s 0us/step


Compile Keras Model

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Fit Keras Model

In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stop]
)


Epoch 1/100
6/6 [==============================] - 376s 61s/step - loss: 40.8600 - accuracy: 0.1620 - val_loss: 12.2682 - val_accuracy: 0.2000
Epoch 2/100
6/6 [==============================] - 372s 63s/step - loss: 12.1440 - accuracy: 0.1006 - val_loss: 2.0095 - val_accuracy: 0.1556
Epoch 3/100
6/6 [==============================] - 367s 62s/step - loss: 2.0218 - accuracy: 0.1508 - val_loss: 1.9501 - val_accuracy: 0.1556
Epoch 4/100
6/6 [==============================] - 352s 59s/step - loss: 2.0544 - accuracy: 0.1564 - val_loss: 1.9459 - val_accuracy: 0.2000
Epoch 5/100
6/6 [==============================] - 366s 61s/step - loss: 2.0276 - accuracy: 0.1844 - val_loss: 1.9569 - val_accuracy: 0.1556
Epoch 6/100
6/6 [==============================] - 370s 61s/step - loss: 1.9413 - accuracy: 0.1732 - val_loss: 1.9497 - val_accuracy: 0.1333
Epoch 7/100
6/6 [==============================] - 368s 62s/step - loss: 1.9461 - accuracy: 0.1453 - val_loss: 1.9515 - val_accuracy: 0.1333
Epoch 8/10

Evaluate Keras Model

In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy*100:.2f}%")


2/2 [==============================] - 24s 8s/step - loss: 1.9459 - accuracy: 0.2000
Validation Loss: 1.9459
Validation Accuracy: 20.00%


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")


2/2 [==============================] - 28s 12s/step - loss: 1.9465 - accuracy: 0.1250
Test Loss: 1.9465
Test Accuracy: 12.50%


Identify Disease from Test Images

In [ ]:
def predict_disease(image_path):
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0) / 255.0
    prediction = model.predict(image)
    predicted_label = np.argmax(prediction)
    return diseases[predicted_label]

# Example usage
test_image_path = '/content/Soybean-/Soybean Leaf Dataset for Disease Classification/Rust/Rust (45).jpg'  # Replace with the path to your test image
predicted_disease = predict_disease(test_image_path)
print(f"Predicted Disease: {predicted_disease}")


1/1 [==============================] - 1s 646ms/step
Predicted Disease: Sudden Death Syndrome


#GoogLeNet

Load and Preprocess Data

In [ ]:
import numpy as np
import os
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# Define the diseases
diseases = ['Bacterial Pustule', 'Frogeye Leaf Spot', 'Healthy', 'Rust', 'Sudden Death Syndrome', 'Target Leaf Spot', 'Yellow Mosaic']

# Initialize lists to store image data and labels
images = []
labels = []

# Load 40 images from each disease for training and validation
# and 10 images for testing
for idx, disease in enumerate(diseases):
    disease_path = os.path.join('/content/Soybean-/Soybean Leaf Dataset for Disease Classification', disease)  # Replace with the path to your dataset
    image_files = [os.path.join(disease_path, f) for f in os.listdir(disease_path) if f.endswith('.jpg')]

    # Split the images into training and validation sets (40 images) and testing set (10 images)
    train_val_images = image_files[:40]
    test_images = image_files[40:]

    for image_file in train_val_images:
        image = load_img(image_file, target_size=(224, 224))  # Resize image to 224x224
        image = img_to_array(image)
        images.append(image)
        labels.append(idx)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generate augmented images
augmented_images = []
augmented_labels = []
for image, label in zip(images, labels):
    image = np.expand_dims(image, axis=0)  # Expand dimensions to (1, height, width, channels) for flow() method
    for batch in datagen.flow(image, batch_size=1):
        augmented_image = batch[0]
        augmented_images.append(augmented_image)
        augmented_labels.append(label)
        break  # Break the loop after one augmentation to avoid infinite loop

# Convert lists to numpy arrays
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# Combine original and augmented data
X_train_val = np.concatenate((images, augmented_images), axis=0)
y_train_val = np.concatenate((labels, augmented_labels), axis=0)

# Shuffle the data
shuffle_idx = np.random.permutation(len(X_train_val))
X_train_val = X_train_val[shuffle_idx]
y_train_val = y_train_val[shuffle_idx]

# Normalize the images
X_train_val = X_train_val / 255.0

# Split the dataset into training, validation, and testing sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)
X_test, y_test = np.array([]), np.array([])  # No test data in this approach


Define GoogLeNet Model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.applications import InceptionV3

def create_googlenet_model(input_shape):
    googlenet_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

    model = Sequential()
    model.add(googlenet_model)

    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(diseases), activation='softmax'))

    return model

input_shape = (224, 224, 3)  # GoogLeNet input shape
model = create_googlenet_model(input_shape)


Compile Keras Model

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Fit Keras Model

In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stop]
)


Epoch 1/100
14/14 [==============================] - 290s 19s/step - loss: 17.3340 - accuracy: 0.1629 - val_loss: 3635.5405 - val_accuracy: 0.1518
Epoch 2/100
14/14 [==============================] - 267s 19s/step - loss: 3.1142 - accuracy: 0.1384 - val_loss: 809.7969 - val_accuracy: 0.1875
Epoch 3/100
14/14 [==============================] - 267s 19s/step - loss: 2.1801 - accuracy: 0.1763 - val_loss: 9024.6191 - val_accuracy: 0.1875
Epoch 4/100
14/14 [==============================] - 272s 20s/step - loss: 2.2013 - accuracy: 0.2388 - val_loss: 458.3075 - val_accuracy: 0.1875
Epoch 5/100
14/14 [==============================] - 285s 20s/step - loss: 2.0348 - accuracy: 0.1763 - val_loss: 49.4507 - val_accuracy: 0.0982
Epoch 6/100
14/14 [==============================] - 287s 21s/step - loss: 2.1347 - accuracy: 0.2031 - val_loss: 15.0587 - val_accuracy: 0.0982
Epoch 7/100
14/14 [==============================] - 294s 21s/step - loss: 2.1853 - accuracy: 0.2210 - val_loss: 6.7742 - val_acc

Evaluate Keras Model

In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy*100:.2f}%")


4/4 [==============================] - 18s 5s/step - loss: 2.0771 - accuracy: 0.2857
Validation Loss: 2.0771
Validation Accuracy: 28.57%


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")


ValueError: Expected input data to be non-empty.

Identify Disease from Test Images

In [ ]:
def predict_disease(image_path):
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0) / 255.0
    prediction = model.predict(image)
    predicted_label = np.argmax(prediction)
    return diseases[predicted_label]


test_image_path = '/content/Soybean-/Soybean Leaf Dataset for Disease Classification/Yellow Mosaic/YM (48).jpg'  # Replace with the path to your test image
predicted_disease = predict_disease(test_image_path)
print(f"Predicted Disease: {predicted_disease}")


1/1 [==============================] - 0s 216ms/step
Predicted Disease: Yellow Mosaic


#RESNET

Load and Preprocess Data

In [ ]:
import numpy as np
import os
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# Define the diseases
diseases = ['Bacterial Pustule', 'Frogeye Leaf Spot', 'Healthy', 'Rust', 'Sudden Death Syndrome', 'Target Leaf Spot', 'Yellow Mosaic']

# Initialize lists to store image data and labels
images = []
labels = []

# Load 40 images from each disease for training and validation
# and 10 images for testing
for idx, disease in enumerate(diseases):
    disease_path = os.path.join('/content/Soybean-/Soybean Leaf Dataset for Disease Classification', disease)  # Replace with the path to your dataset
    image_files = [os.path.join(disease_path, f) for f in os.listdir(disease_path) if f.endswith('.jpg')]

    # Split the images into training and validation sets (40 images) and testing set (10 images)
    train_val_images = image_files[:40]
    test_images = image_files[40:]

    for image_file in train_val_images:
        image = load_img(image_file, target_size=(224, 224))  # Resize image to 224x224
        image = img_to_array(image)
        images.append(image)
        labels.append(idx)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images by 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20% of the width
    height_shift_range=0.2,  # Randomly shift images vertically by 20% of the height
    shear_range=0.2,  # Shear intensity (shear angle in radians)
    zoom_range=0.2,  # Randomly zoom images by 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill mode for filling in newly created pixels
)

# Generate augmented images
augmented_images = []
augmented_labels = []
for image, label in zip(images, labels):
    image = np.expand_dims(image, axis=0)  # Expand dimensions to (1, height, width, channels) for flow() method
    for batch in datagen.flow(image, batch_size=1):
        augmented_image = batch[0]
        augmented_images.append(augmented_image)
        augmented_labels.append(label)
        break  # Break the loop after one augmentation to avoid infinite loop

# Convert lists to numpy arrays
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# Combine original and augmented data
X_train_val = np.concatenate((images, augmented_images), axis=0)
y_train_val = np.concatenate((labels, augmented_labels), axis=0)

# Shuffle the data
shuffle_idx = np.random.permutation(len(X_train_val))
X_train_val = X_train_val[shuffle_idx]
y_train_val = y_train_val[shuffle_idx]

# Normalize the images
X_train_val = X_train_val / 255.0

# Split the dataset into training, validation, and testing sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)
X_test, y_test = np.array([]), np.array([])  # No test data in this approach


Define ResNet50 Model

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.applications import ResNet50
from keras.optimizers import Adam

def create_resnet50_model(input_shape):
    resnet50_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    model = Sequential()
    model.add(resnet50_model)

    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(diseases), activation='softmax'))

    return model

input_shape = (224, 224, 3)  # ResNet50 input shape
model = create_resnet50_model(input_shape)


Compile Keras Model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Fit Keras Model

In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stop]
)


Epoch 1/100


KeyboardInterrupt: 

Evaluate Keras Model

In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy*100:.2f}%")


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

Identify Disease from Test Images

In [ ]:
def predict_disease(image_path):
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0) / 255.0
    prediction = model.predict(image)
    predicted_label = np.argmax(prediction)
    return diseases[predicted_label]

test_image_path = '/content/Soybean-/Soybean Leaf Dataset for Disease Classification/Yellow Mosaic/YM (48).jpg'  # Replace with the path to your test image
predicted_disease = predict_disease(test_image_path)
print(f"Predicted Disease: {predicted_disease}")
